# Classificador de Documentos: Advertisement vs Scientific Article

Este notebook implementa regras de processamento de imagens para diferenciar documentos do tipo **advertisement** de **scientific_article** usando o dataset RVL-CDIP.


## 1. Instalação e Importação de Bibliotecas


In [ ]:
# Instalação de dependências necessárias
!pip install opencv-python-headless numpy pillow matplotlib scikit-image scikit-learn scipy


In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


## 2. Classe de Extração de Características

### Características Distintivas:

**Advertisement:**
- Maior quantidade de cores
- Imagens e gráficos coloridos
- Layout não uniforme
- Texto em tamanhos variados
- Alta saturação de cores

**Scientific Article:**
- Predominantemente texto em preto/branco
- Layout estruturado (colunas)
- Baixa variação de cor
- Texto uniforme
- Possíveis gráficos/tabelas em preto e branco


In [ ]:
class DocumentFeatureExtractor:
    """
    Extrator de características visuais para classificação de documentos.
    """
    
    def __init__(self):
        pass
    
    def load_image(self, image_path):
        """
        Carrega imagem em diferentes formatos.
        """
        img = cv2.imread(str(image_path))
        if img is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return img
    
    def extract_color_features(self, img):
        """
        Extrai características relacionadas a cores.
        Advertisements tendem a ter mais cores e maior saturação.
        """
        # Converter para HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Calcular saturação média
        saturation_mean = np.mean(hsv[:, :, 1])
        saturation_std = np.std(hsv[:, :, 1])
        
        # Calcular diversidade de cores usando histograma
        hist_h = cv2.calcHist([hsv], [0], None, [180], [0, 180])
        hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])
        
        # Entropia do histograma (maior entropia = mais cores diferentes)
        hist_h_norm = hist_h / (hist_h.sum() + 1e-7)
        color_entropy = -np.sum(hist_h_norm * np.log2(hist_h_norm + 1e-7))
        
        # Contagem de cores únicas (simplificado)
        img_resized = cv2.resize(img, (100, 100))
        unique_colors = len(np.unique(img_resized.reshape(-1, 3), axis=0))
        
        return {
            'saturation_mean': saturation_mean,
            'saturation_std': saturation_std,
            'color_entropy': color_entropy,
            'unique_colors': unique_colors
        }
    
    def extract_text_density(self, img):
        """
        Analisa densidade de texto.
        Scientific articles têm alta densidade de texto uniforme.
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Binarização
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Densidade de pixels pretos (texto)
        text_density = np.sum(binary > 0) / binary.size
        
        # Análise de componentes conectados (palavras/blocos de texto)
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)
        
        # Filtrar componentes pequenos (ruído)
        min_area = 50
        text_components = [stat for stat in stats[1:] if stat[cv2.CC_STAT_AREA] > min_area]
        
        return {
            'text_density': text_density,
            'num_text_components': len(text_components),
            'avg_component_size': np.mean([s[cv2.CC_STAT_AREA] for s in text_components]) if text_components else 0
        }
    
    def extract_layout_features(self, img):
        """
        Analisa características de layout.
        Scientific articles têm layout estruturado em colunas.
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Projeção horizontal (soma de pixels por linha)
        horizontal_projection = np.sum(gray < 128, axis=1)
        
        # Projeção vertical (soma de pixels por coluna)
        vertical_projection = np.sum(gray < 128, axis=0)
        
        # Variância das projeções (artigos científicos têm padrões mais regulares)
        h_proj_variance = np.var(horizontal_projection)
        v_proj_variance = np.var(vertical_projection)
        
        # Detectar colunas através de vales na projeção vertical
        v_proj_smoothed = cv2.GaussianBlur(vertical_projection.astype(float).reshape(-1, 1), (15, 1), 0).flatten()
        v_proj_mean = np.mean(v_proj_smoothed)
        
        # Contar transições significativas (possíveis colunas)
        threshold = v_proj_mean * 0.5
        below_threshold = v_proj_smoothed < threshold
        transitions = np.sum(np.diff(below_threshold.astype(int)) != 0)
        
        return {
            'h_proj_variance': h_proj_variance,
            'v_proj_variance': v_proj_variance,
            'layout_transitions': transitions
        }
    
    def extract_edge_features(self, img):
        """
        Analisa características de bordas.
        Advertisements têm mais bordas devido a imagens e gráficos.
        """
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detectar bordas com Canny
        edges = cv2.Canny(gray, 50, 150)
        
        # Densidade de bordas
        edge_density = np.sum(edges > 0) / edges.size
        
        # Detectar linhas com transformada de Hough
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)
        num_lines = len(lines) if lines is not None else 0
        
        return {
            'edge_density': edge_density,
            'num_lines': num_lines
        }
    
    def extract_all_features(self, image_path):
        """
        Extrai todas as características de uma imagem.
        """
        img = self.load_image(image_path)
        
        features = {}
        features.update(self.extract_color_features(img))
        features.update(self.extract_text_density(img))
        features.update(self.extract_layout_features(img))
        features.update(self.extract_edge_features(img))
        
        return features


## 3. Classificador Baseado em Regras


In [ ]:
class DocumentClassifier:
    """
    Classificador baseado em regras para diferenciar Advertisement de Scientific Article.
    """
    
    def __init__(self):
        self.feature_extractor = DocumentFeatureExtractor()
        
        # Thresholds calibrados para o dataset RVL-CDIP
        self.thresholds = {
            'saturation_mean': 30,  # Ads têm saturação > 30
            'color_entropy': 5.0,   # Ads têm entropia > 5.0
            'text_density': 0.15,   # Scientific articles têm densidade > 0.15
            'unique_colors': 2000,  # Ads têm mais cores únicas
        }
    
    def calculate_score(self, features):
        """
        Calcula score para classificação.
        Score positivo = Advertisement
        Score negativo = Scientific Article
        """
        score = 0
        
        # Regra 1: Alta saturação sugere Advertisement
        if features['saturation_mean'] > self.thresholds['saturation_mean']:
            score += 2
        else:
            score -= 1
        
        # Regra 2: Alta entropia de cores sugere Advertisement
        if features['color_entropy'] > self.thresholds['color_entropy']:
            score += 2
        else:
            score -= 1
        
        # Regra 3: Alta densidade de texto sugere Scientific Article
        if features['text_density'] > self.thresholds['text_density']:
            score -= 2
        else:
            score += 1
        
        # Regra 4: Muitas cores únicas sugere Advertisement
        if features['unique_colors'] > self.thresholds['unique_colors']:
            score += 1
        else:
            score -= 1
        
        # Regra 5: Layout estruturado (baixa variância vertical) sugere Scientific Article
        if features['v_proj_variance'] < 1e9:
            score -= 1
        
        # Regra 6: Muitas transições de layout sugerem colunas (Scientific Article)
        if features['layout_transitions'] > 10:
            score -= 1
        
        return score
    
    def classify(self, image_path):
        """
        Classifica uma imagem como Advertisement ou Scientific Article.
        """
        features = self.feature_extractor.extract_all_features(image_path)
        score = self.calculate_score(features)
        
        classification = 'advertisement' if score > 0 else 'scientific_article'
        confidence = abs(score) / 10.0  # Normalizar para 0-1
        
        return {
            'classification': classification,
            'score': score,
            'confidence': min(confidence, 1.0),
            'features': features
        }
    
    def visualize_classification(self, image_path, result):
        """
        Visualiza o resultado da classificação.
        """
        img = cv2.imread(str(image_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Imagem original
        axes[0].imshow(img_rgb)
        axes[0].set_title(f"Documento Original", fontsize=14)
        axes[0].axis('off')
        
        # Resultados
        axes[1].axis('off')
        result_text = f"""
        CLASSIFICAÇÃO: {result['classification'].upper()}
        
        Score: {result['score']}
        Confiança: {result['confidence']:.2%}
        
        CARACTERÍSTICAS EXTRAÍDAS:
        
        Cores:
        - Saturação média: {result['features']['saturation_mean']:.2f}
        - Entropia de cores: {result['features']['color_entropy']:.2f}
        - Cores únicas: {result['features']['unique_colors']}
        
        Texto:
        - Densidade: {result['features']['text_density']:.4f}
        - Componentes: {result['features']['num_text_components']}
        
        Layout:
        - Transições: {result['features']['layout_transitions']}
        - Var. Vertical: {result['features']['v_proj_variance']:.2e}
        
        Bordas:
        - Densidade: {result['features']['edge_density']:.4f}
        - Linhas detectadas: {result['features']['num_lines']}
        """
        
        axes[1].text(0.1, 0.5, result_text, fontsize=11, family='monospace',
                    verticalalignment='center')
        
        plt.tight_layout()
        plt.show()


## 4. Exemplo de Uso

### 4.1 Configuração do Dataset

Para usar o dataset RVL-CDIP:
1. Baixe o dataset do Kaggle: https://www.kaggle.com/datasets/pdavpoojan/the-rvlcdip-dataset-test
2. Extraia os arquivos
3. Configure o caminho abaixo


In [ ]:
# Configure o caminho do dataset aqui
DATASET_PATH = Path('/path/to/rvl-cdip-test')  # Ajuste conforme necessário

# O dataset RVL-CDIP tem a seguinte estrutura:
# - advertisement/
# - scientific_article/  (ou scientific_publication/)
# etc.


### 4.2 Classificar uma Imagem Individual


In [ ]:
# Inicializar classificador
classifier = DocumentClassifier()

# Exemplo: classificar uma imagem
# image_path = DATASET_PATH / 'advertisement' / 'image_001.png'
# result = classifier.classify(image_path)
# classifier.visualize_classification(image_path, result)


### 4.3 Testar com Imagem de Exemplo


In [ ]:
def test_classifier_on_sample(image_path):
    """
    Testa o classificador em uma imagem e mostra os resultados.
    """
    classifier = DocumentClassifier()
    result = classifier.classify(image_path)
    
    print("="*60)
    print(f"Imagem: {Path(image_path).name}")
    print("="*60)
    print(f"Classificação: {result['classification'].upper()}")
    print(f"Score: {result['score']}")
    print(f"Confiança: {result['confidence']:.2%}")
    print("="*60)
    
    classifier.visualize_classification(image_path, result)
    
    return result

# Exemplo de uso:
# test_classifier_on_sample('path/to/your/image.png')


### 4.4 Avaliação em Múltiplas Imagens


In [ ]:
def evaluate_classifier(ad_folder, article_folder, num_samples=10):
    """
    Avalia o classificador em um conjunto de imagens.
    """
    classifier = DocumentClassifier()
    
    results = {
        'advertisement': {'correct': 0, 'total': 0},
        'scientific_article': {'correct': 0, 'total': 0}
    }
    
    # Testar advertisements
    ad_images = list(Path(ad_folder).glob('*.png'))[:num_samples]
    for img_path in ad_images:
        try:
            result = classifier.classify(img_path)
            results['advertisement']['total'] += 1
            if result['classification'] == 'advertisement':
                results['advertisement']['correct'] += 1
            print(f"✓ {img_path.name}: {result['classification']} (score: {result['score']})")
        except Exception as e:
            print(f"✗ Erro em {img_path.name}: {e}")
    
    # Testar scientific articles
    article_images = list(Path(article_folder).glob('*.png'))[:num_samples]
    for img_path in article_images:
        try:
            result = classifier.classify(img_path)
            results['scientific_article']['total'] += 1
            if result['classification'] == 'scientific_article':
                results['scientific_article']['correct'] += 1
            print(f"✓ {img_path.name}: {result['classification']} (score: {result['score']})")
        except Exception as e:
            print(f"✗ Erro em {img_path.name}: {e}")
    
    # Calcular métricas
    print("\n" + "="*60)
    print("RESULTADOS DA AVALIAÇÃO")
    print("="*60)
    
    for doc_type, metrics in results.items():
        if metrics['total'] > 0:
            accuracy = metrics['correct'] / metrics['total']
            print(f"\n{doc_type.upper()}:")
            print(f"  Corretas: {metrics['correct']}/{metrics['total']}")
            print(f"  Acurácia: {accuracy:.2%}")
    
    total_correct = sum(m['correct'] for m in results.values())
    total = sum(m['total'] for m in results.values())
    overall_accuracy = total_correct / total if total > 0 else 0
    
    print(f"\nACURÁCIA GERAL: {overall_accuracy:.2%}")
    print("="*60)
    
    return results

# Exemplo de uso:
# evaluate_classifier(
#     ad_folder='path/to/advertisement',
#     article_folder='path/to/scientific_article',
#     num_samples=20
# )


## 5. Ajuste Fino dos Thresholds (Opcional)


In [ ]:
def analyze_feature_distribution(ad_folder, article_folder, num_samples=20):
    """
    Analisa a distribuição de características para ajustar thresholds.
    """
    extractor = DocumentFeatureExtractor()
    
    ad_features = []
    article_features = []
    
    # Coletar features de advertisements
    for img_path in list(Path(ad_folder).glob('*.png'))[:num_samples]:
        try:
            features = extractor.extract_all_features(img_path)
            ad_features.append(features)
        except:
            pass
    
    # Coletar features de scientific articles
    for img_path in list(Path(article_folder).glob('*.png'))[:num_samples]:
        try:
            features = extractor.extract_all_features(img_path)
            article_features.append(features)
        except:
            pass
    
    # Visualizar distribuições
    feature_names = ['saturation_mean', 'color_entropy', 'text_density', 'unique_colors']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, feature_name in enumerate(feature_names):
        ad_values = [f[feature_name] for f in ad_features]
        article_values = [f[feature_name] for f in article_features]
        
        axes[idx].hist(ad_values, alpha=0.5, label='Advertisement', bins=20, color='red')
        axes[idx].hist(article_values, alpha=0.5, label='Scientific Article', bins=20, color='blue')
        axes[idx].set_xlabel(feature_name)
        axes[idx].set_ylabel('Frequência')
        axes[idx].legend()
        axes[idx].set_title(f'Distribuição: {feature_name}')
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas
    print("\nESTATÍSTICAS DAS CARACTERÍSTICAS:")
    print("="*80)
    for feature_name in feature_names:
        ad_values = [f[feature_name] for f in ad_features]
        article_values = [f[feature_name] for f in article_features]
        
        print(f"\n{feature_name}:")
        print(f"  Advertisement    - Média: {np.mean(ad_values):.2f}, Std: {np.std(ad_values):.2f}")
        print(f"  Scientific Article - Média: {np.mean(article_values):.2f}, Std: {np.std(article_values):.2f}")

# Exemplo de uso:
# analyze_feature_distribution(
#     ad_folder='path/to/advertisement',
#     article_folder='path/to/scientific_article',
#     num_samples=30
# )


## 6. Resumo das Regras de Classificação

### Regras Implementadas:

1. **Saturação de Cores**: Advertisements têm cores mais saturadas (média > 30)
2. **Entropia de Cores**: Advertisements têm maior diversidade de cores (entropia > 5.0)
3. **Densidade de Texto**: Scientific articles têm maior densidade de texto (> 0.15)
4. **Cores Únicas**: Advertisements têm mais cores únicas (> 2000 em imagem 100x100)
5. **Layout Estruturado**: Scientific articles têm layout mais uniforme
6. **Transições de Layout**: Scientific articles frequentemente têm colunas (> 10 transições)

### Como Melhorar:

- Coletar estatísticas do seu dataset específico usando `analyze_feature_distribution()`
- Ajustar os thresholds em `DocumentClassifier.thresholds`
- Adicionar mais regras baseadas em características específicas observadas
- Considerar usar Machine Learning para classificação automática
